In [1]:
!pip install -q --upgrade pip
!pip install jaxlib==0.4.4
!pip install -q --upgrade jax[cuda11_cudnn805] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q git+https://www.github.com/google/neural-tangents
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
!pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cpu.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.8/112.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 24.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cpu
Looking in indexe

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK


In [4]:
import neural_tangents as nt
from neural_tangents import stax
from jax import numpy as np
from jax import random
import jax

from utils import *
from layers import index_aggregation

# spars edge lists two graphs example test the stax.Aggregate behaviour

In [5]:
# define a toy 4x4 graph
# 0----3
# |    |
# 1----2
# and 2x2 graph
# 0----1

edge_list_1 = np.array([[0,1],
                      [0,3],
                      [1,0],
                      [1,2],
                      [2,1],
                      [2,3],
                      [3,2],
                      [3,0]])

node_features_1 = np.array([[1,2],
                          [11,12],
                          [21,22],
                          [31,32]])

node_features_1 = node_features_1 * 1.0

edge_list_2 = np.array([[0,1],
                        [1,0],
                        [0,0],
                        [1,1]]) + 4

node_features_2 = np.array([[1,2],
                          [21,22]])

node_features_2 = node_features_2 * 1.0 

graph_indx = np.append(np.full(node_features_1.shape[0], 0), np.full(node_features_2.shape[0], 1), 0)
edge_list = np.append(edge_list_1, edge_list_2, 0)
node_features = np.append(node_features_1, node_features_2, 0)

print(graph_indx.shape)
print(edge_list.shape)
print(node_features.shape)

node_features = np.expand_dims(node_features, 0)
node_features = np.expand_dims(node_features, 2)

# need this to be able to aggregat over both axis, which we need for the kernel function to work
edge_list = np.expand_dims(edge_list, 0)
edge_list = np.expand_dims(edge_list, 2)
edge_list = np.append(edge_list, edge_list, axis=2)

print(edge_list.shape)
print(node_features.shape)

edge_list

(6,)
(12, 2)
(6, 2)
(1, 12, 2, 2)
(1, 6, 1, 2)


Array([[[[0, 1],
         [0, 1]],

        [[0, 3],
         [0, 3]],

        [[1, 0],
         [1, 0]],

        [[1, 2],
         [1, 2]],

        [[2, 1],
         [2, 1]],

        [[2, 3],
         [2, 3]],

        [[3, 2],
         [3, 2]],

        [[3, 0],
         [3, 0]],

        [[4, 5],
         [4, 5]],

        [[5, 4],
         [5, 4]],

        [[4, 4],
         [4, 4]],

        [[5, 5],
         [5, 5]]]], dtype=int32)

In [6]:
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Conv(5, (1,1), parameterization="standard"),
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE"),
)

key = random.PRNGKey(0)
key, subkey = jax.random.split(key)
_, params = init_fn(subkey, node_features.shape)

# set all weights to 1 for the convolution
params = [(np.ones(params[0][0].shape), None), (), ()]

/usr/local/lib/python3.9/dist-packages/neural_tangents/_src/stax/linear.py:439: UserWarning: Negative indices in `pattern` are considered as padding (i.e. ignored), unlike typical numpy negative indexing.
  warnings.warn('Negative indices in `pattern` are considered as padding '


In [7]:
out = apply_fn(params, node_features, pattern=edge_list)
print(out.shape)
np.squeeze(out)

(1, 6, 1, 5)


Array([[86., 86., 86., 86., 86.],
       [46., 46., 46., 46., 46.],
       [86., 86., 86., 86., 86.],
       [46., 46., 46., 46., 46.],
       [46., 46., 46., 46., 46.],
       [46., 46., 46., 46., 46.]], dtype=float32)

In [8]:
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Conv(5, (1,1), parameterization="standard"),
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE"),
    stax.Conv(5, (1,1), parameterization="standard"),
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE"),
)


k = kernel_fn(node_features, node_features, ('ntk'), pattern=(edge_list, edge_list))
np.squeeze(k).shape

/usr/local/lib/python3.9/dist-packages/neural_tangents/_src/stax/linear.py:439: UserWarning: Negative indices in `pattern` are considered as padding (i.e. ignored), unlike typical numpy negative indexing.
  warnings.warn('Negative indices in `pattern` are considered as padding '


(6, 6)

# spars edge lists two graphs example test the stax.Aggregate and the index_aggregation behaviour (Test the apply function)

In [9]:
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Conv(5, (1,1), parameterization="standard"),
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE"),
    index_aggregation(),
    # stax.Dense(1)
)

key = random.PRNGKey(0)
key, subkey = jax.random.split(key)
_, params = init_fn(subkey, node_features.shape)
params

/usr/local/lib/python3.9/dist-packages/neural_tangents/_src/stax/linear.py:439: UserWarning: Negative indices in `pattern` are considered as padding (i.e. ignored), unlike typical numpy negative indexing.
  warnings.warn('Negative indices in `pattern` are considered as padding '


[(Array([[[[-1.2358007 ,  1.0902201 , -0.17007037, -0.12051605,
             0.8951071 ],
           [ 0.20728225, -0.6802286 ,  0.16402271,  0.60656357,
             0.7451222 ]]]], dtype=float32), None), (), ()]

In [10]:
# set all weights to 1 for the convolution
params[0] = (np.ones(params[0][0].shape), None)
params

[(Array([[[[1., 1., 1., 1., 1.],
           [1., 1., 1., 1., 1.]]]], dtype=float32), None), (), ()]

In [11]:
# we have two graphs and the last conv layer as 5 channels. 
out = apply_fn(params, node_features, pattern=edge_list, graph_indx=graph_indx)
print(out.shape)
np.squeeze(out)

(2, 5)


Array([[264., 264., 264., 264., 264.],
       [ 92.,  92.,  92.,  92.,  92.]], dtype=float32)

## Test the kernel function

In [14]:
# we have 6 edges -> bevore aggreagtion, the kernel is 6x6
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Conv(5, (1,1), parameterization="standard"),
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE"),
    # index_aggregation(),
    # stax.Dense(1)
)

k = kernel_fn(node_features, node_features, ('ntk'), pattern=(edge_list, edge_list), graph_indx=graph_indx)
print(np.squeeze(k).shape)
np.squeeze(k)

/usr/local/lib/python3.9/dist-packages/neural_tangents/_src/stax/linear.py:439: UserWarning: Negative indices in `pattern` are considered as padding (i.e. ignored), unlike typical numpy negative indexing.
  warnings.warn('Negative indices in `pattern` are considered as padding '


(6, 6)


Array([[3700., 1980., 3700., 1980., 1980., 1980.],
       [1980., 1060., 1980., 1060., 1060., 1060.],
       [3700., 1980., 3700., 1980., 1980., 1980.],
       [1980., 1060., 1980., 1060., 1060., 1060.],
       [1980., 1060., 1980., 1060., 1060., 1060.],
       [1980., 1060., 1980., 1060., 1060., 1060.]], dtype=float32)

In [15]:
# this is the expected result of the kernel function of the index_aggregation
# the input has 2 graphs => the kernel is a 2x2 array
# the layer bevor the aggregation is 6x6 (because 6 nodes in input)
# the first 4 nodes are from graph one => the sum of the 4x4 upper left block matrix is the "self kernel" of graph 1
# the las 2 nodes are from graph two => the sum of the 2x2 lower right block matrix is the "self kernel" of graph 2
# the ramaining 2x4 and 4x2 block matricies are the kernel of graph 1 graph 2 and vice versa
kernel = np.squeeze(k)
print(np.sum(kernel[:4,:4]))
print(np.sum(kernel[-2:,-2:]))
print(np.sum(kernel[:4,-2:]))
print(np.sum(kernel[:-2,4:]))

34880.0
4240.0
12160.0
12160.0


In [16]:
# now with the aggregation
# results match the expectation
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Conv(5, (1,1), parameterization="standard"),
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE"),
    index_aggregation(),
    # stax.Dense(1)
)

k = kernel_fn(node_features, node_features, ('ntk'), pattern=(edge_list, edge_list), graph_indx=(graph_indx, graph_indx), nb_graphs=(2,2))
k

Array([[[[34880., 12160.],
         [12160.,  4240.]]]], dtype=float32)

In [18]:
# now with dense layer after the aggregation
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Conv(5, (1,1), parameterization="standard"),
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE"),
    index_aggregation(),
    stax.Dense(1)
)

k = kernel_fn(node_features, node_features, ('ntk'), pattern=(edge_list, edge_list), graph_indx=(graph_indx, graph_indx), nb_graphs=(2,2))
k

Array([[[[52320., 18240.],
         [18240.,  6360.]]]], dtype=float32)